In [5]:
import os

In [6]:
%pwd

'c:\\Users\\gokul\\OneDrive\\Desktop\\Projects\\Fashion_Recommender_with_MLOps_EndtoEnd\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'c:\\Users\\gokul\\OneDrive\\Desktop\\Projects\\Fashion_Recommender_with_MLOps_EndtoEnd'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_weights: str

In [10]:
from myFashionRecommender.constants import *
from myFashionRecommender.utils.common import read_yaml,create_directories

In [24]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:

        config = self.config.prepare_base_model

        create_directories([config.root_dir])
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            params_image_size = self.params.IMAGE_SIZE,
            params_include_top = self.params.INCLUDE_TOP,
            params_weights = self.params.WEIGHTS
        )
        return prepare_base_model_config


In [25]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


In [26]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.resnet50.ResNet50(
            include_top = self.config.params_include_top,
            weights = self.config.params_weights,
            input_shape = self.config.params_image_size
        )
        self.model.trainable = False
        
        self.save_model(path=self.config.base_model_path,model = self.model)
    
    @staticmethod
    def _prepare_full_model(base_model):
        model = tf.keras.Sequential(
            [
                base_model,
                tf.keras.layers.GlobalMaxPooling2D()
            ]
        )
        model.summary()
        return model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            base_model= self.model
        )
        self.save_model(path=self.config.updated_base_model_path,model=self.full_model)
    
    @staticmethod
    def save_model(path: Path,model: tf.keras.Model):
        model.save(path)

In [27]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-07-05 00:36:22,621: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 00:36:22,624: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 00:36:22,625: INFO: common: created directory at: artifacts]
[2024-07-05 00:36:22,626: INFO: common: created directory at: artifacts/prepare_base_model]
[2024-07-05 00:36:24,505: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 global_max_pooling2d (Globa  (None, 2048)             0         
 lMaxPooling2D)                                                  
                                           